#4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%reset-f
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy

In [ ]:
def cal_mean(insts,n):
  m= len(insts[:,0])
  mean = np.ones(n)
  for i in range(n):
    mean[i] = insts[:,i].mean()
  return mean

def cal_sigma(insts,mean):
  [m,n] = insts.shape
  
  sigma = (insts - mean) ** 2
  sigma = np.sum(sigma,axis =0)
  sigma = sigma/m
  return sigma

def update(insts,mean,m,n,k):
  l = np.ones(m)
  for i in range(m):
    val = []
    for j in range(k):
      val.append(np.sum((insts[i,0:n]- mean[j])**2))
    l[i] = np.argmin(val) + 1
  return l
def performance(mat,m):
  for i in range(3):
    ia = mat[i,i]/(mat[i,0]+mat[i,1]+mat[i,2])
    print('accuracy for group is',end='')
    print(i+1, end=': ')
    print(ia)
  print('overall accuracy is',end= ' ')
  print((mat[0,0]+mat[1,1]+mat[2,2])/m)

def hot_enc(insts):
  y_en = np.zeros([len(insts[:,0]),3])
  y_en[:,0][insts[:,-1]==1] = 1
  y_en[:,1][insts[:,-1]==2] = 1
  y_en[:,2][insts[:,-1]==3] = 1
  return y_en

In [ ]:
def k_mean(insts,k):
  [m,n] = insts.shape
  y = np.random.randint(k,size = (m,1))+1
  insts = np.append(insts,y,axis = 1)

  itr = 100
  L_prev = insts[:,-1]
  for i in range(itr):
  
    mean_vec = []
    for j in range(k):
      insts_j = insts[insts[:,-1] == j+1]
      mean_j = cal_mean(insts_j,n)
      mean_vec.append(mean_j)
    L_curr = update(insts,mean_vec,m,n,k)
    insts[:,-1] = L_curr
    if np.array_equal(L_curr,L_prev): #termination
      sigma_vec = []
      for j in range(k):
        insts_j = insts[insts[:,-1] == j+1]
        sigma_j = cal_sigma(insts_j[:,0:n],mean_vec[j])
        sigma_vec.append(sigma_j)
      return sigma_vec,mean_vec
      
    L_prev = L_curr

def gaussian(x,center,beta):
    p = np.exp(-beta * (np.linalg.norm(x - center)) ** 2)
    return p 

def cubic(x, center):
    return (np.linalg.norm(x - center))**3

In [ ]:
path = '/content/drive/MyDrive/ML/data5.xlsx'
df = pd.read_excel(path)
insts = df.to_numpy() #number of instances
m= len(insts[:,0])
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n= len(insts[0,:])-1 # number of features
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std() #Normalize the data
np.random.shuffle(insts)

insts_train= insts[0:int(m*0.7),:]
insts_test= insts[int(m*0.7):int(m*0.9),:]
insts_val = insts[int(m*0.9):m,:]

m_train= len(insts_tr[:,0])
m_val = len(insts_val[:,0]) 
m_test= len(insts_te[:,0])

n1  =  (int)(m/(2*(n+3)))
n2  =  (int)(m/(2*(n1+3)))

itrain= 1
print(n1,n2)
y_train= hot_enc(insts_tr)
y_test= hot_enc(insts_te)
y_val = hot_enc(insts_val)

#RADIAL BASIS FUNCTION NEURAL NETWORK
k=5
sigma,mean = k_mean(insts_tr[:,0:n],k)
mean = np.array(mean)
sigma = np.array(sigma)
beta =0.5/(sigma**2 + 1e-6)
H = np.zeros((m_tr,k))
for i in range(m_tr):
  for j in range(k):
    H[i,j] = cubic(insts_tr[i,0:n],mean[j])

print(H)

w = np.dot(np.linalg.pinv(H),y_tr)
H_test= np.zeros((m_te,k))
for i in range(m_te):
  for j in range(k):
    H_te[i,j] = cubic(insts_te[i,0:n],mean[j])
y_pre = np.dot(H_te,w)
y_p = np.zeros(m_te)
for i in range(m_te):
  y_p[i] = np.argmax(y_pre[i])+1

In [ ]:
confmat = pd.crosstab(insts_te[:,-1], y_p)
confmat = np.asarray(confmat)
performance(confmat,m_te)

accuracy for group1: 0.7857142857142857
accuracy for group2: 1.0
accuracy for group3: 0.8333333333333334
overall accuracy: 0.8809523809523809
